## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np
import sys

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value=int(sys.argv[1])

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 44.6 ms ± 63 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
And the result of the sum of numbers in the range [0, value) is: 24966.22672115008

Time taken by reduction operation using numpy.sum(): 91.7 µs ± 797 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.sum(): 24966.22672114992 
 
Time taken by reduction operation using numpy.ndarray.sum(): 73 µs ± 193 ns per loop (mean ± std. dev. of 2 runs, 10,000 loops each)
Now, the result using numpy.ndarray.sum(): 24966.22672114992


a) Librería multiprocessing: En la siguiente celda de código del notebook4 vas a utilizar el paquete
multiprocessing para acelerar la operación de reducción. Para ello, importa Pool de la librería
multiprocessing, y con la función map llama a la función reduc_operation creando solo un proceso
con el tamaño del array original de [0, value], creando 2 procesos llamándolos con 2 arrays que tienen
la copia de los valores del array original pero de tamaños [0, int(value/2)] y [int(value/2), value)], y
creando 4 procesos que llaman a la función con 4 arrays que tienen la copia de los valores del array
original pero de tamaños [0, int(value/4)], [int(value/4), int(value/2)], [int(value/2), int(3*value/4)]y [int(3*value/4), value]. Como verás, el tiempo para calcular la suma va disminuyendocada vezque duplicas el número de procesos (cores) que usas.


In [3]:
from multiprocessing import Pool
import numpy as np
import time
#recuperamos la funcion de reduc_operation
def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s
X = np.random.rand(value)#volvemos a crear el array entre 0-1 con una longitud de value
#para separar los arrays necesitamos una funcion que reciba el array y el numero de particiones
def separador(array, partes):
    tamaño_subarrays=len(array)//partes
    lista_arrays=[]#la idea es tener listas de arrays para luego aplicarle a todas la funcion reduc_operation con maop
    for i in range (partes):
        inicio=i*tamaño_subarrays
        if i==partes-1:#siempre debemos considerar que la iteracion para la lista finaliza un número de n-partes divididas porque empezamos la iteracion en el 0
            fin=len(array)
        else:
            fin=(i+1)*tamaño_subarrays
        lista_arrays.append(array[inicio:fin])

    return lista_arrays

def paralelo(array,n_procesos):
    lista_subarrays=separador(array,n_procesos)#usamos la funcion separador para dividir el array antes de trabajar con el
    with Pool(processes=n_procesos) as pool:
        #map nos hace aplicar una funcion a cada elemento de una lista
        suma_subarray=pool.map(reduc_operation,lista_subarrays)#con map no hay que llamar a al funcion directamente y pasarle parámetros
        return np.sum(suma_subarray)

#Vamos a resolver el apartado A para 1,2 y 4 procesos y arrays del tamaño de cada subproceso

for i in [1,2,4]:
    tiempoini=time.time()
    suma=paralelo(X,i)
    tiempofin=time.time()
    print(f"la suma total para {i}proceso es {suma} con un tiempo de {tiempofin-tiempoini} segundos")
    


la suma total para 1proceso es 24999757.935457647 con un tiempo de 48.190600633621216 segundos
la suma total para 2proceso es 24999757.935450844 con un tiempo de 26.063174724578857 segundos
la suma total para 4proceso es 24999757.935453597 con un tiempo de 13.571947813034058 segundos


b) Libreria Numba: En la siguiente celda de código vamos a utilizar el paquete numba. Importa de
dicho paquete njit y prange, y haz 2 nuevas versiones de la función original reduc_operation(A)
decorándolas con el decorador @njit y @njit(parallel=True)5
respectivamente. Como ya vimos en la
práctica anterior, solo con utilizar @njit se mejora mucho el tiempo de ejecución, pero si además
ejecutamos en paralelo aún se nota una mayor reducción de tiempo.


In [13]:
from numba import njit, prange
import numpy as np
#version 1 con njit
@njit
def reduc_operation_secuencial(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s
    
@njit(parallel=True)
def reduc_operation_paralela(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s


X = np.random.rand(value)

tiempo1 = %timeit -r 2 -o -q reduc_operation_secuencial(X)
tiempo2 = %timeit -r 2 -o -q reduc_operation_paralela(X)

print("Time taken by reduction operation using a function:", tiempo1)
print("Time taken by reduction operation using a function:", tiempo2)

Time taken by reduction operation using a function: 196 ms ± 1.28 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Time taken by reduction operation using a function: 15.2 ms ± 3.34 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


#APARTADO D pego la salida de mi sbatch:
Numero de la variable value 50000000
Time taken by reduction operation using a function: 45.2 s ± 89.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 24998354.356284916

Time taken by reduction operation using numpy.sum(): 60.5 ms ± 419 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.sum(): 24998354.356286045 
 
Time taken by reduction operation using numpy.ndarray.sum(): 60.7 ms ± 826 µs per loop (mean ± std. dev. of 2 runs, 10 loops each)
Now, the result using numpy.ndarray.sum(): 24998354.356286045
la suma total para 1proceso es 24998071.18308363 con un tiempo de 47.24898386001587 segundos
la suma total para 2proceso es 24998071.18307965 con un tiempo de 25.767412662506104 segundos
la suma total para 4proceso es 24998071.183079787 con un tiempo de 13.36642861366272 segundos
Time taken by reduction operation using a function: 197 ms ± 1.51 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
Time taken by reduction operation using a function: 19.2 ms ± 6.33 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

#EXPLICACION Y ANALISIS:

-Respecto a multiprocessing debemos tener en cuenta que  utilizamos python  interpretando linea a linea el código, observamos que la mejora con el paralelismo prácticamente es un x2 cuando dividimos "el doble" los procesos de la tarea entre núcleos. La mejora no es muy interesante cuando usamos de esta forma python e implementamos el paralelismo.

-Respecto a numba y numpy conjuntas, observamos una mejora considerable. Dentro de estas dos versiones donde gracias a njit ya compilamos parte del código y nos permite óptimizar enormemente el tiempo de ejecución:
njit sin paralelismo es buena mejora, unas 230 veces más rápida que  la versión inicial de nuestro código{reduct operation}.
njit con paralelismo activado es 2300 veces más rapidas que la version de reduct operation(ganadora).

-Sin embargo, cuando reobservamos los resultados con el uso de numpy vemos que numpy.sum  con  60.5 ms, es muy buena opción cuando la comparamos con el uso de numba. Pero al combinar numba con procesos en paralelo y numpy, obtenemos una optimización en el tiempo de ejecución que la convirtiendose en la candidata favorita.

